# Polar coding for any $N$ with array for speeding up
We develop the simplist Polar coding example assuming the length of code is $N$=2. This will be used for AI example using Tensorflow. We will see how new rx method imitate Polar.
- input을 꼬아 주어야 한다. 앞쪽에서 뺼때는 꼬여있어야 하기 때문이다.

## Common functions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import numba as nb
import matplotlib.pyplot as plt

In [3]:
from wireless import nb_polar

In [8]:
nb_polar.polar_bec(2)

array([0.75, 0.25])

In [9]:
nb_polar.polar_bec(4)

array([0.9375, 0.4375, 0.5625, 0.0625])

In [94]:
def _polar_bec(N_code=4, erase_prob=0.5):
    """
    BEC에 대해 Polar code의 예측 성능을 구한다. 단, 비트당 제거 오류율은 erase_prob로 가정한다.
    """
    n = int(np.log2(N_code))
    E = np.zeros(N_code)
    # E_out = np.zeros(N_code)    
    E[0] = erase_prob
    for i in range(n):
        LN = 2**i
        # print(i, LN)
        # print('E in:', E)
        # i stage에서 끝은 LN*2이다. 안그러면 broadcast되어 버린다.
        E[LN:LN*2] = E[:LN] * E[:LN]
        E[:LN] = 1-(1-E[:LN])*(1-E[:LN])   
        # print('E out:', E)        
    return E

@nb.jit
def polar_bec(N_code=4, erase_prob=0.5):
    """
    BEC에 대해 Polar code의 예측 성능을 구한다. 단, 비트당 제거 오류율은 erase_prob로 가정한다.
    """
    n = int(np.log2(N_code))
    E = np.zeros(N_code)
    # E_out = np.zeros(N_code)    
    E[0] = erase_prob
    for i in range(n):
        LN = 2**i
        # print(i, LN)
        # print('E in:', E)
        # i stage에서 끝은 LN*2이다. 안그러면 broadcast되어 버린다.
        E[LN:LN*2] = E[:LN] * E[:LN]
        E[:LN] = 1-(1-E[:LN])*(1-E[:LN])   
        # print('E out:', E)        
    return E

In [95]:
%time polar_bec(2048, 0.1)

CPU times: user 1.16 s, sys: 15.6 ms, total: 1.17 s
Wall time: 1.22 s


array([1.        , 0.99996608, 0.99999999, ..., 0.        , 0.        ,
       0.        ])

In [96]:
%time polar_bec(2048, 0.1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 77 µs


array([1.        , 0.99996608, 0.99999999, ..., 0.        , 0.        ,
       0.        ])

In [97]:
%time _polar_bec(2048, 0.1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.14 ms


array([1.        , 0.99996608, 0.99999999, ..., 0.        , 0.        ,
       0.        ])

In [98]:
nb_polar.polar_design_bec(2,1)

BER for each bit [0.75 0.25]


array([1, 0])

In [99]:
nb_polar.polar_design_bec(4,2)

BER for each bit [0.9375 0.4375 0.5625 0.0625]


array([1, 0, 1, 0])

In [100]:
nb_polar.polar_design_bec(8,4)

BER for each bit [0.99609375 0.68359375 0.80859375 0.12109375 0.87890625 0.19140625
 0.31640625 0.00390625]


array([1, 1, 1, 0, 1, 0, 0, 0])

In [101]:
nb_polar.polar_design_bec(8,4,0.1)

BER for each bit [5.6953279e-01 3.9403990e-02 7.0896790e-02 1.9999000e-04 1.1826721e-01
 3.9601000e-04 1.3032100e-03 1.0000000e-08]


array([1, 1, 1, 0, 1, 0, 0, 0])